In [ ]:
!pip install gensim
!pip install scipy==1.12
!pip install janome  # janomeをインストール

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from janome.tokenizer import Tokenizer

# 日本語テキストのトークン化を行うTokenizerの設定
tokenizer = Tokenizer()

def tokenize(text):
    """テキストをトークン化し、意味を持つ単語（名詞、動詞、形容詞）のみを抽出してリストとして返します。"""
    tokens = tokenizer.tokenize(text)
    return [token.surface for token in tokens if token.part_of_speech.split(',')[0] in ['名詞', '動詞', '形容詞']]

# サンプルデータセットの定義
documents = [
    "ディープラーニングは人工知能の一機能です",
    "人工知能と機械学習はコンピュータサイエンスの一部です",
    "ディープラーニングはほとんどのAIアプリケーションで使用されています"
]

tagged_data = [
    TaggedDocument(words=tokenize(doc), tags=[str(i)])
    for i, doc in enumerate(documents)
]

# Doc2Vecモデルのインスタンス作成と設定
model = Doc2Vec(vector_size=20, min_count=1, epochs=10)

# モデルの語彙を構築
model.build_vocab(tagged_data)

# モデルの訓練
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# 新しいクエリ文書をトークン化
query = "AIとディープラーニング"
query_tokens = tokenize(query)

# クエリ文書のベクトルを推論
query_vector = model.infer_vector(query_tokens)

# クエリと類似した文書を検索
similar_docs = model.dv.most_similar(positive=[query_vector], topn=len(documents))

#===============================================
# 出力
for i, doc in enumerate(documents):
    tokens = tokenize(doc)
    print(f"対象文書{i}: {doc}")
    print(f"\t分かち書きの結果（トークン化） => {tokens}\n")

print(f"\nクエリ文書: {query}")
query_tokens = tokenize(query)
print(f"\t分かち書きの結果（トークン化） => {query_tokens}\n")

print("==============================================================")
# 訓練された文書ベクトルを表示
print("訓練された文書ベクトル:")
for i in range(len(tagged_data)):
    print(f"Document {i}: Vector: {model.dv[str(i)]}")

print("\n推論されたクエリベクトル:")
print(f"Query Vector: {query_vector}")

# 類似した文書を検索した結果
print("\n類似文書とその類似度:")
for doc_id, similarity in similar_docs:
    print(f"文書 {doc_id}: {documents[int(doc_id)]}, 類似度: {similarity:.4f}")